In [1]:
#imports
import joblib
import cv2
import numpy as np

In [2]:
model=joblib.load('Face_joblib')

In [3]:
face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_classifier=cv2.CascadeClassifier('haarcascade_eye.xml')

In [4]:
def get_cropped_image_if_2_eyes(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_classifier.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color

In [5]:

import pywt
def w2d(img, mode='haar', level=5):
    imArr = img
    #convert to grayscale
    imArr = cv2.cvtColor( imArr,cv2.COLOR_BGR2GRAY )
    #convert to float
    imArr =  np.float32(imArr)   
    imArr /= 255;
    # compute coefficients 
    coeffs=pywt.wavedec2(imArr, mode, level=level)
    #Process Coefficients to get features from image
    coeffs_H=list(coeffs) 
    coeffs_H[0] *= 0;
    
    # reconstruction
    imArr_H=pywt.waverec2(coeffs_H, mode);
    imArr_H *= 255;
    imArr_H =  np.uint8(imArr_H)

    return imArr_H



In [6]:
class_dict={'Milan': 0, 'Aditya': 1, 'lionel_messi': 2, 'Bhargav': 3, 'Hareesh': 4}
class_dict

{'Milan': 0, 'Aditya': 1, 'lionel_messi': 2, 'Bhargav': 3, 'Hareesh': 4}

In [7]:
cap = cv2.VideoCapture(0) #webcam

while True:
    ret,frame = cap.read()
    if ret == False:
        continue
    faces = face_classifier.detectMultiScale(frame,1.3,5)
    if(len(faces)==0):
        continue
    for face in faces:
        x,y,w,h = face
        #Get the face ROI and preprocessing on Live image
        offset = 10
        face_section = frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section = cv2.resize(face_section,(32,32))
        img_har_live =w2d(face_section,'haar',5)
        scalled_img_har_live = cv2.resize(img_har_live, (32, 32))
        combined_img_live = np.vstack((face_section.reshape(32*32*3,1),scalled_img_har_live.reshape(32*32,1)))
        X_live=[]
        X_live.append(combined_img_live)
        X_live=np.array(X_live).reshape(len(X_live),4096).astype(float)
        X_live
        #prediction on live image
        m_live=model.predict(X_live)
        m_problive=model.predict_proba(X_live)
        Pred_prob_live=round(m_problive.max()*100,2)
        Pred_prob_live_string = str(Pred_prob_live)
        Pred_value_live=m_live[0]
        for Name_live in class_dict.keys(): 
            if(class_dict.get(Name_live)==Pred_value_live):
                Pred_Name_live=Name_live
        #placing prediction name, probability on live webfeed
        cv2.putText(frame,Pred_Name_live,(x,y-10),cv2.FONT_ITALIC,1,(255,255,0),2,cv2.LINE_AA)
        cv2.putText(frame,Pred_prob_live_string,(x,y-30),cv2.FONT_ITALIC,1,(255,255,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)

    cv2.imshow("Faces",frame)
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

END